In [ ]:
import pyodbc
import pandas as pd

# Database connection information
server = r'DEVADMIN08-LAP\AYMAN'
database = 'item_Dwh_clean'
username = 'm_ayman'
password = '123456'

# Establish a connection to SQL Server
conn = pyodbc.connect(
    f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'
)

# SQL query to fetch all records from the view "vw_item_raw2"
query2 = "SELECT * FROM vw_item_raw2"

# Execute the query and load results into a Pandas DataFrame
cleaned = pd.read_sql(query2, conn)

# Close the database connection after finishing
conn.close()


C:\Users\mohamed.ayman\AppData\Local\Temp\2\ipykernel_6464\262922999.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  not_cleaned = pd.read_sql(query1, conn)
C:\Users\mohamed.ayman\AppData\Local\Temp\2\ipykernel_6464\262922999.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  cleaned = pd.read_sql(query2, conn)


In [2]:
# check the column
cleaned.columns

Index(['itemeanID_sk', 'itemclass_bk', 'itemclass_name_ar',
       'itemclass_name_en', 'section_bk', 'section_name_ar', 'section_name_en',
       'section_usage', 'section_transdate', 'itemgroup_bk',
       'itemgroup_name_ar', 'itemgroup_name_en', 'itemgroup_transdate',
       'group_expirycheck', 'department_bk', 'itemsubgroup',
       'subgroup_name_ar', 'subgroup_name_en', 'subgroup_expirycheck',
       'itemean_bk', 'barcode_item_unit_bk', 'itemean_name_ar',
       'flavour_item_description', 'unit_bk', 'peices', 'unit_name_ar',
       'unit_name_en', 'unitprice', 'costprice', 'retailprice',
       'old_costprice', 'oldunitprice', 'Modified_date', 'store_class_bk',
       'store_class_name_ar', 'store_class_name_en', 'itemean_name_en',
       'item_status', 'item_userid', 'transid', 'item_transdate',
       'item_sales_tax', 'item_modified_date', 'item_isSalesOnline',
       'Supplier_no_FK', 'userid_item_unit', 'transid_item_unit',
       'transdate_item_unit', 'IsSalesOnLine', 

In [3]:
# choose columns
df = cleaned[['itemean_name_ar','department_bk','itemclass_bk','section_bk','itemgroup_bk','itemsubgroup','costprice','retailprice','unitprice']].copy()

In [ ]:
# 2) Remove rows with missing values (NaN) in either of the two columns
df = df.dropna(subset=['itemean_name_ar', 'department_bk'])

# Count rows after dropping NaN values
after_dropna = len(df)

# Calculate number of removed rows
removed_rows = initial_rows - after_dropna

# Calculate percentage of removed rows
removed_percentage = (removed_rows / initial_rows) * 100

# Print summary
print(f"بعد حذف NaN: {after_dropna} (حذف {removed_rows} صف - {removed_percentage:.2f}%)")


الصفوف الكلية: 1383631


In [ ]:
# 2) Remove rows with missing values (NaN) in either 'itemean_name_ar' or 'department_bk'
df = df.dropna(subset=['itemean_name_ar', 'department_bk'])

# Get the number of rows after dropping NaN values
after_dropna = len(df)

# Print summary: how many rows remain and how many were removed
print(f"بعد حذف NaN: {after_dropna} (حذف {initial_rows - after_dropna} صف)")


بعد حذف NaN: 1383631 (حذف 0 صف)


In [ ]:
# 3) Check for duplicate rows (before cleaning)
# Count duplicates where both 'itemean_name_ar' and 'department_bk' are the same
dups_before = df.duplicated(subset=['itemean_name_ar', 'department_bk']).sum()

# Print how many duplicate rows were found
print(f"عدد الصفوف المكررة (نفس الاسم + نفس المجموعة) قبل أي تنظيف: {dups_before}")


عدد الصفوف المكررة (نفس الاسم + نفس المجموعة) قبل أي تنظيف: 1247076


In [ ]:
# Show examples of duplicate rows if any exist
if dups_before > 0:
    print("أمثلة على تكرارات (أول 5):")
    
    # Select rows that are marked as duplicates (keep=False keeps all occurrences)
    duplicates = df[df.duplicated(subset=['itemean_name_ar', 'department_bk'], keep=False)]
    
    # Display the first 10 duplicate rows
    print(duplicates.head(10))


أمثلة على تكرارات (أول 5):
                    itemean_name_ar  department_bk  itemclass_bk  section_bk  \
0                    مكعبات سودانى             129             1          27   
1         بيزاريلا جبنه بلوك بالوزن           2902             1           2   
2           مسن سكاكين وسط  KH-2530           8935             2          14   
3  (1H.TEL)بنطلون جينز سليم م 28-32           3000             2          24   
4            شنطة باك سبيدرمان20276           3108             2          24   
5  فاز عربيه طمطم اغانى كود:1220004           6629             2          12   
6        فرشه متعدد زاويه كارت 8393           5949             2           9   
7      فانوس صوفيا فستان NO:LD-131D           6721             2          12   
8      فريش فارم لانشون ساده بالوزن           2903             1           2   
9              قطوف تفاح جولدن اصفر            113             1          16   

   itemgroup_bk  itemsubgroup  costprice  retailprice  unitprice  
0           221          

In [ ]:
# 4) Remove direct duplicates 
# (same 'itemean_name_ar' and same 'department_bk' repeated multiple times)
df = df.drop_duplicates(subset=['itemean_name_ar', 'department_bk']).reset_index(drop=True)

# Count rows after removing duplicates
after_dropdup = len(df)

# Print summary showing how many rows remain and how many duplicates were removed
print(f"بعد حذف التكرارات المباشرة: {after_dropdup} (حذف {after_dropna - after_dropdup} صف)")


بعد حذف التكرارات المباشرة: 136555 (حذف 1247076 صف)


In [ ]:
# Define a list of excluded groups (these groups will be removed from the dataset)
excluded_groups = [199, 150, 70, 7219, 77, 5, 7207, 3, 191, 7232, 999, 207, 36, 227, 200, 205, 222]

# Filter the DataFrame: keep only rows where 'itemgroup_bk' is NOT in the excluded list
df = df[~df['itemgroup_bk'].isin(excluded_groups)]


In [ ]:
import re
import pandas as pd

# Custom list of Arabic stopwords + colors
arabic_stopwords = {
    # Common words
    'في','على','من','إلى','الى','و','فيه','مع','منه','عن','هذا','هذه','ذلك',
    'الذي','التي','ما','هو','هي','كان','كانت','قد','لن','لم','لا','إن','ان','أن',
    'كل','أو','ثم','حتى','كما','هناك','بعد','قبل','أكثر','أقل','مثل','بين','أي','ايا','موضوع',

    # Business-related words
    'لتر','وسط','بالوزن','رجالي','مل','طقم','جم','م','هم','وزن','قطع','ق','ص',

    # Color names
    'ابيض','اسود','احمر','ازرق','اخضر','اصفر','برتقالي','بنفسجي','بني','رمادي','فضي','ذهبي',
    'موف','كحلي','وردي','سماوي','تركواز','ليموني','زيتي','بيج'
}

# Regex for Arabic diacritics (harakat)
arabic_diacritics = re.compile(r'[\u0617-\u061A\u064B-\u0652]')

def clean_arabic_text(text: str) -> str:
    """Clean Arabic text by removing diacritics, stopwords, and unwanted characters"""
    if pd.isna(text):
        return ''
    text = str(text).strip()

    # Remove diacritics (harakat)
    text = re.sub(arabic_diacritics, '', text)
    
    # Normalize Arabic letters
    text = re.sub(r'[إأآ]', 'ا', text)
    text = re.sub(r'ى', 'ي', text)
    text = re.sub(r'ؤ', 'و', text)
    text = re.sub(r'ئ', 'ي', text)
    text = re.sub(r'ة', 'ه', text)
    
    # Remove Tatweel (ـ)
    text = text.replace('ـ', '')
    
    # Remove non-Arabic characters (keep only Arabic letters and spaces)
    text = re.sub(r'[^ء-ي\s]', ' ', text)
    
    # Reduce multiple spaces
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Remove stopwords and very short words
    tokens = [t for t in text.split() if (t not in arabic_stopwords and len(t) > 1)]
    return ' '.join(tokens)

# Apply cleaning on 'itemean_name_ar' column
df['itemean_name_ar_clean'] = df['itemean_name_ar'].apply(clean_arabic_text)

# Create additional features
df['name_length'] = df['itemean_name_ar_clean'].str.len()                  # length of cleaned name
df['word_count'] = df['itemean_name_ar_clean'].str.split().str.len()       # number of words
split_words = df['itemean_name_ar_clean'].str.split()
df['first_word'] = split_words.str[0]                                      # first word
df['last_word'] = split_words.str[-1]                                      # last word
df['has_number'] = df['itemean_name_ar_clean'].str.con_]()_]()


In [11]:
df[['itemean_name_ar','itemean_name_ar_clean','department_bk']]

,itemean_name_ar,itemean_name_ar_clean,department_bk
0,مكعبات سودانى,مكعبات سوداني,129
1,بيزاريلا جبنه بلوك بالوزن,بيزاريلا جبنه بلوك,2902
2,مسن سكاكين وسط KH-2530,مسن سكاكين,8935
3,(1H.TEL)بنطلون جينز سليم م 28-32,بنطلون جينز سليم,3000
4,شنطة باك سبيدرمان20276,شنطه باك سبيدرمان,3108
...,...,...,...
136550,لوك اند لوك لانش بوكس مقسم1.5لتر (ازرق)HPL782,لوك اند لوك لانش بوكس مقسم,9686
136551,EVE6 دوركو ماكينة حلاقه حريمي 6 شفره 3+1هديه,دوركو ماكينه حلاقه حريمي شفره هديه,9722
136552,لوك اند لوك لانش بوكس مقسم980مل (ازرق) HPL781,لوك اند لوك لانش بوكس مقسم,9686
136553,طقم ملاحه ستانلس *2ق على ستاند 553/ qq-296,ملاحه ستانلس علي ستاند,8219


In [ ]:
# 6) Show distribution of groups (top 20) 
# This helps to check if the dataset is imbalanced
print("\nأعلى 20 مجموعة (department_bk) حسب عدد العناصر:")

# Count occurrences of each department and show the top 20
print(df['department_bk'].value_counts().head(20))



أعلى 20 مجموعة (department_bk) حسب عدد العناصر:
department_bk
9679    2620
6701    1590
6714    1442
6708    1343
7833     984
6721     964
8002     935
6718     844
6706     833
8201     816
8001     791
7903     777
6622     770
7301     755
7602     733
6422     714
3109     709
6738     706
6702     700
3004     694
Name: count, dtype: int64


In [13]:
df[[
    'itemean_name_ar_clean','first_word','last_word','itemclass_bk','section_bk','itemsubgroup','department_bk',   
    'costprice', 'retailprice', 'unitprice',
      'name_length', 'word_count', 'has_number']].head()

,itemean_name_ar_clean,first_word,last_word,itemclass_bk,section_bk,itemsubgroup,department_bk,costprice,retailprice,unitprice,name_length,word_count,has_number
0,مكعبات سوداني,مكعبات,سوداني,1,27,129,129,0.01,44.0,44.0,13,2,0
1,بيزاريلا جبنه بلوك,بيزاريلا,بلوك,1,2,2922,2902,45.00,52.0,52.0,18,3,0
2,مسن سكاكين,مسن,سكاكين,2,14,8955,8935,8.00,10.0,10.0,10,2,0
3,بنطلون جينز سليم,بنطلون,سليم,2,24,3000,3000,45.00,55.0,55.0,16,3,0
4,شنطه باك سبيدرمان,شنطه,سبيدرمان,2,24,3110,3108,65.00,75.0,75.0,17,3,0


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# 1.1) Remove rare classes (departments) that have very few samples

# Count how many rows exist for each department
counts = df['department_bk'].value_counts()

# Keep only departments that have at least 4 rows
valid_classes = counts[counts >= 4].index

# Filter the DataFrame to include only valid departments
df = df[df['department_bk'].isin(valid_classes)]

# Print summary: number of rows left and number of unique departments remaining
print(f"بعد إزالة الفئات النادرة: {len(df)} صف")
print(f"عدد الفئات المتبقية: {df['department_bk'].nunique()}")


بعد إزالة الفئات النادرة: 130017 صف
عدد الفئات المتبقية: 1340


### Preparation for models

In [ ]:
# Features (text) and target (labels)
X = df["itemean_name_ar_clean"]  # Series of cleaned product names
y = df["department_bk"]          # Target: department/category

# Initialize TF-IDF vectorizer
# max_features=5000 limits to top 5000 features
# ngram_range=(1, 2) considers both unigrams (single words) and bigrams (pairs of words)
tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))

# Fit the TF-IDF vectorizer on X and transform the text into numerical features
X_tfidf = tfidf.fit_transform(X)


In [ ]:
from imblearn.over_sampling import BorderlineSMOTE

# Initialize Borderline-SMOTE for oversampling minority classes
# random_state=42 for reproducibility
# k_neighbors=1 specifies the number of nearest neighbors to use
smote = BorderlineSMOTE(random_state=42, k_neighbors=1)

# Apply Borderline-SMOTE to balance the dataset
# X_tfidf: features, y: target labels
X_resampled, y_resampled = smote.fit_resample(X_tfidf, y)

# Print the shapes before and after resampling
print("قبل:", X_tfidf.shape, y.shape)
print("بعد :", X_resampled.shape, y_resampled.shape)


قبل: (130017, 5000) (130017,)
بعد : (3446039, 5000) (3446039,)


# MultinomialNB 


In [ ]:
from sklearn.model_selection import train_test_split

# Split the resampled dataset into training and testing sets
# test_size=0.2 means 20% of the data will be used for testing
# stratify=y_resampled ensures that class distribution is preserved in both sets
# random_state=42 for reproducibility
X_train, X_test, y_train, y_test = train_test_split(
    X_resampled,
    y_resampled,
    test_size=0.2,
    stratify=y_resampled,
    random_state=42
)


In [ ]:
import numpy as np
from sklearn.naive_bayes import MultinomialNB

# Get the list of unique classes in the training set
classes = np.unique(y_train)

# Create equal class priors for all classes
# Each class gets the same probability: 1 / number of classes
equal_priors = np.ones(len(classes)) / len(classes)

# Initialize Multinomial Naive Bayes model with equal class priors
model = MultinomialNB(class_prior=equal_priors)

# Fit the model on the training data
model.fit(X_train, y_train)


MultinomialNB(class_prior=array([0.00074627, 0.00074627, 0.00074627, ..., 0.00074627, 0.00074627,
       0.00074627], shape=(1340,)))

In [ ]:
# 6) Make predictions on the test set
y_pred = model.predict(X_test)  # Predict the department for each item in X_test


In [ ]:
from sklearn.metrics import classification_report

# 7) Evaluate model performance
print("\n📊 تقرير الأداء:")

# Print precision, recall, F1-score, and support for each class
print(classification_report(y_test, y_pred))



📊 تقرير الأداء:


C:\Users\mohamed.ayman\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

           1       0.41      1.00      0.58       524
           3       0.98      0.85      0.91       524
           5       0.99      1.00      1.00       524
           7       0.00      0.00      0.00         1
          12       0.75      0.51      0.61       524
          13       0.85      0.75      0.80       524
          14       0.77      0.98      0.86       524
          15       0.93      0.90      0.92       524
          16       0.65      0.91      0.75       524
          17       0.78      0.38      0.51       524
          18       0.86      1.00      0.93       524
          19       1.00      1.00      1.00       524
          20       0.97      0.91      0.94       524
          21       1.00      0.70      0.82        10
          22       0.74      0.97      0.84       524
          23       0.92      0.96      0.94       524
          24       0.96      0.84      0.90       524
          25       0.00    

C:\Users\mohamed.ayman\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\mohamed.ayman\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## 🧾 Model Evaluation Summary

- **Accuracy:** 0.88  
- **Macro Average:**
  - Precision: 0.88  
  - Recall: 0.87  
  - F1-Score: 0.87  
- **Weighted Average:**
  - Precision: 0.89  
  - Recall: 0.88  
  - F1-Score: 0.88  
- **Total Samples Evaluated:** 689,208  

✅ The model achieves **88% accuracy**, with strong balance between precision, recall, and F1-score across classes.


# LogisticRegression

In [ ]:
import numpy as np

# 1) Compute class distribution in the training set
class_counts = y_train.value_counts()

# 2) Identify rare classes (those with less than 2 samples)
rare_classes = class_counts[class_counts < 2].index

# 3) Create a boolean mask (True for rows to keep, False for rare classes)
mask = ~y_train.isin(rare_classes).to_numpy()

# 4) Filter the Sparse Matrix (X_train) and target (y_train) using the mask
X_train_filtered = X_train[mask]
y_train_filtered = y_train[mask]


In [ ]:
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# 1) Build the pipeline
# - Step 1: Apply SMOTE to oversample minority classes and handle class imbalance
# - Step 2: Use Logistic Regression with class_weight='balanced' to give higher weight to underrepresented classes
pipeline = Pipeline([
    ('smote', SMOTE(random_state=42, k_neighbors=1)),  
    ('logreg', LogisticRegression(
        max_iter=1000,
        class_weight='balanced',  
        solver='lbfgs',
        multi_class='auto'
    ))
])

# 2) Train the pipeline on the filtered training set
pipeline.fit(X_train_filtered, y_train_filtered)

# 3) Make predictions on the test set
y_pred = pipeline.predict(X_test)


C:\Users\mohamed.ayman\AppData\Roaming\Python\Python313\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


In [ ]:
# 4) Evaluate the model performance
# Print a classification report (precision, recall, f1-score, support for each class)
print(classification_report(y_test, y_pred))


C:\Users\mohamed.ayman\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

           1       1.00      1.00      1.00       524
           3       0.97      1.00      0.98       524
           5       1.00      1.00      1.00       524
           7       1.00      1.00      1.00         1
          12       0.81      0.64      0.71       524
          13       0.86      0.96      0.91       524
          14       0.96      0.97      0.96       524
          15       0.98      0.98      0.98       524
          16       0.73      0.90      0.81       524
          17       0.84      0.69      0.75       524
          18       1.00      1.00      1.00       524
          19       1.00      1.00      1.00       524
          20       0.98      0.91      0.94       524
          21       0.90      0.90      0.90        10
          22       1.00      0.95      0.97       524
          23       0.98      0.97      0.97       524
          24       0.82      0.99      0.89       524
          25       1.00    

C:\Users\mohamed.ayman\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\mohamed.ayman\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## 🧾 Model Evaluation Summary

- **Accuracy:** 0.95  
- **Macro Average:**
  - Precision: 0.95  
  - Recall: 0.94  
  - F1-Score: 0.94  
- **Weighted Average:**
  - Precision: 0.95  
  - Recall: 0.95  
  - F1-Score: 0.95  
- **Total Samples Evaluated:** 689,208  

✅ The model achieves **95% accuracy**, showing excellent performance with very high precision, recall, and F1-scores across all classes.


-----------------------------------------------------------------------------------------------------

# Test

In [ ]:
def predict_section(user_text, model, tfidf, cleaned_df):
    """
    Takes user input -> cleans it -> predicts department_bk -> 
    returns code + group name + class name + section name + subgroup name
    """
    # 1) Clean the input text
    cleaned_text = clean_arabic_text(user_text).lower().strip()
    
    if not cleaned_text:
        return None, None, None, None, None, "⚠️ Invalid input after cleaning"
    
    # 2) Convert text to TF-IDF features
    text_tfidf = tfidf.transform([cleaned_text])
    
    # 3) Predict the department code
    predicted_class = model.predict(text_tfidf)[0]
    
    # 4) Fetch corresponding names (Group + Class + Section + Subgroup)
    row = cleaned_df.loc[
        cleaned_df['department_bk'] == predicted_class, 
        ['itemgroup_name_ar', 'itemclass_name_ar', 'section_name_ar','subgroup_name_ar']
    ].dropna().head(1)
    
    if not row.empty:
        group_name    = row['itemgroup_name_ar'].values[0]
        class_name    = row['itemclass_name_ar'].values[0]
        section_name  = row['section_name_ar'].values[0]
        subgroup_name = row['subgroup_name_ar'].values[0] 
        return predicted_class, group_name, class_name, section_name, subgroup_name, None
    else:
        return predicted_class, "غير معروف", "غير معروف", "غير معروف", "غير معروف", None


# --- Test with user input ---
user_input = input("Enter product name: ")

predicted_class, group_name, class_name, section_name, subgroup_name, error_msg = predict_section(
    user_input, pipeline, tfidf, cleaned
)

if error_msg:
    print(error_msg)
else:
    print(f"🔮 Product Name: {user_input}")
    print(f"🏷️ Predicted Class: {class_name}")
    print(f"📂 Predicted Section: {section_name}")
    print(f"📦 Main Group: {group_name}")
    print(f"🛢 Subgroup: {subgroup_name}")


🔮 Product Name: لمبة ليد 9 وات
🏷️ الكلاس المتوقع: لا غذائى
📂 السيكشن المتوقع: أدوات كهربائية
📦 المجموعة الرئيسية: مستلزمات أدوات كهربائيه
🛢 المجموعة الفرعية: لمبات ليد


# Test with propabilty

In [ ]:
def predict_topn(user_text, model, tfidf, cleaned_df, top_n=3):
    """
    Takes user input -> cleans it -> gets the top N predicted classes from the model
    Returns (class_id + group + class + section + subgroup + probability)
    """
    # 1) Clean the input text (remove unwanted chars, lowercase, strip spaces)
    cleaned_text = clean_arabic_text(user_text).lower().strip()
    
    if not cleaned_text:
        # If text is empty after cleaning -> return an error message
        return None, "⚠️ Invalid input after cleaning"
    
    # 2) Convert cleaned text into TF-IDF vector representation
    text_tfidf = tfidf.transform([cleaned_text])
    
    # 3) Predict probability distribution over all possible classes
    probs = model.predict_proba(text_tfidf)[0]
    
    # 4) Sort probabilities in descending order and select top N classes
    top_indices = np.argsort(probs)[::-1][:top_n]
    
    results = []
    for idx in top_indices:
        # Get the predicted class ID
        predicted_class = model.classes_[idx]
        # Get the probability score of this class
        prob = probs[idx]
        
        # 5) Lookup the class details (Group, Class, Section, Subgroup) from cleaned_df
        row = cleaned_df.loc[
            cleaned_df['department_bk'] == predicted_class, 
            ['itemgroup_name_ar', 'itemclass_name_ar', 'section_name_ar', 'subgroup_name_ar']
        ].dropna().head(1)

        if not row.empty:
            group_name    = row['itemgroup_name_ar'].values[0]
            class_name    = row['itemclass_name_ar'].values[0]
            section_name  = row['section_name_ar'].values[0]
            subgroup_name = row['subgroup_name_ar'].values[0]
        else:
            # If class ID not found in DataFrame -> mark as unknown
            group_name, class_name, section_name, subgroup_name = "غير معروف", "غير معروف", "غير معروف", "غير معروف"
        
        # 6) Append the result for this class with its probability
        results.append({
            "class_id": int(predicted_class),
            "group": group_name,
            "class": class_name,
            "section": section_name,
            "subgroup": subgroup_name,
            "probability": round(prob, 3)
        })
    
    return results


In [62]:
# Take input from the user
user_input = input("أدخل اسم المنتج: ")

# Call the prediction function to get the top N predictions
predictions = predict_topn(user_input, pipeline, tfidf, cleaned, top_n=3)

# Check if the function returned an error message (string)
if isinstance(predictions, str):  
    print(predictions)  # Print error message
else:
    # Print the product name entered by the user
    print(f"🔮 Product Name: {user_input}")
    
    # Loop through the predictions and display details
    for i, p in enumerate(predictions, 1):
        print(f"\n📌 Prediction {i}:")
        print(f"   🏷️ Predicted Class: {p['class']}")
        print(f"   📂 Predicted Section: {p['section']}")
        print(f"   📦 Main Group: {p['group']}")
        print(f"   🛢 Subgroup: {p['subgroup']}")
        print(f"   🔢 Class ID: {p['class_id']} (with probability {p['probability']*100:.1f}%)")


🔮 Product Name: كوكو دجاج مجمد

📌 Prediction 1:
   🏷️ Predicted Class: غذائى
   📂 Predicted Section: ثلاجات تجميد
   📦 Main Group: طيور مجمده
   🛢 Subgroup: دجاج كامل
   🔢 Class ID: 1585 (with probability 87.8%)

📌 Prediction 2:
   🏷️ Predicted Class: غذائى
   📂 Predicted Section: ثلاجات تجميد
   📦 Main Group: طيور مجمده
   🛢 Subgroup: اجزاء دجاج
   🔢 Class ID: 1589 (with probability 3.1%)

📌 Prediction 3:
   🏷️ Predicted Class: غذائى
   📂 Predicted Section: ثلاجات تجميد
   📦 Main Group: خضروات مجمده
   🛢 Subgroup: خضار بالوزن
   🔢 Class ID: 1600 (with probability 1.5%)
